<a href="https://colab.research.google.com/github/Ksenia-90/recommender_systems/blob/lesson_2_22/HW_2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install implicit==0.4.4

     |████████████████████████████████| 1.1 MB 12.6 MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-linux_x86_64.whl size=3413742 sha256=643301398f375ee81bf5e9794c419563872be175075fd3f9b55dbd7abf16ebc9
  Stored in directory: /root/.cache/pip/wheels/44/7e/7d/a17324ea207cfbe76aca878b5b8ca0aa932cf55d163329be37
Successfully built implicit


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
data  = pd.read_csv("/content/drive/My Drive/Colab Notebooks/retail_train.csv")
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [ ]:
#data = pd.read_csv('data/retail_train.csv')
#data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [8]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [9]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    recs = np.random.choice(items_weights.item_id,
                            p=items_weights.weight, 
                            size=n,
                            replace=False)
    return recs.tolist()

In [10]:
def get_weight_matrix(data):

    items_weight = data.groupby('item_id')['sales_value'].sum().reset_index()
    log_value = items_weight.sales_value.apply(lambda x: 0 if x < 1 else np.log(x))
    log_value_sum = log_value.sum()
    items_weight['weight'] = log_value.apply(lambda x: x / log_value_sum)

    return items_weight

In [11]:
%%time
items_weight = get_weight_matrix(data)
items_weight.head() 
# your_code

CPU times: user 253 ms, sys: 8.52 ms, total: 262 ms
Wall time: 274 ms


In [12]:
#проверка
items_weight.weight.sum()

0.9999999999999999

## Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
#Копируем часть данных с вебинара

# 1. Бейзлайны

In [15]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


### 1.1 Random recommendation

In [13]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [16]:
%%time

items = data_train.item_id.unique()
result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(2)

CPU times: user 3.16 s, sys: 32.9 ms, total: 3.19 s
Wall time: 3.19 s


### 1.2 Popularity-based recommendation

In [17]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [18]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

CPU times: user 105 ms, sys: 762 µs, total: 106 ms
Wall time: 106 ms


## 2.1 Item-Item Recommender / ItemKNN

In [19]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [20]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [21]:
data_train.head(100)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1060,26985040735,1,9553288,1,8.49,315,0.00,1251,1,0.0,0.0
96,1351,26985052379,1,903230,1,0.99,447,-0.30,1955,1,0.0,0.0
97,744,26985165432,1,5978648,0,0.00,31582,0.00,1119,1,0.0,0.0
98,212,26985205886,1,822346,1,1.25,288,-0.34,1341,1,0.0,0.0


In [22]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 6666
data_train.head(100)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1060,26985040735,1,9553288,1,8.49,315,0.00,1251,1,0.0,0.0
96,1351,26985052379,1,903230,1,0.99,447,-0.30,1955,1,0.0,0.0
97,744,26985165432,1,6666,0,0.00,31582,0.00,1119,1,0.0,0.0
98,212,26985205886,1,822346,1,1.25,288,-0.34,1341,1,0.0,0.0


In [23]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix)


In [24]:
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

5.33770796861036

In [25]:
# создаем словари мапинга между id бизнеса к строчному id матрицы

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [26]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей


model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.83 s, sys: 19.8 ms, total: 1.85 s
Wall time: 1.31 s


In [27]:
recs

[(0, 78679.0),
 (3408, 72173.0),
 (2149, 57995.0),
 (3947, 17272.0),
 (3587, 14417.0)]

In [28]:
[id_to_itemid[rec[0]] for rec in recs]

[6666, 1082185, 981760, 1127831, 1098066]

In [29]:
%%time

result['itemitem'] = result['user_id'].apply(lambda user_id: [id_to_itemid[rec[0]]  
                                                              for rec in model.recommend(userid=userid_to_id[user_id], 
                                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                                N=5, 
                                                                filter_already_liked_items=False, 
                                                                filter_items=None, 
                                                                recalculate_user=True)
                                                                             ])

CPU times: user 53.5 ms, sys: 0 ns, total: 53.5 ms
Wall time: 54 ms


In [31]:
result.head(10)

,user_id,actual,random_recommendation,popular_recommendation,itemitem
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13910406, 13192522, 62804, 1131236, 1042390]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[9803602, 959210, 929171, 13987234, 12525209]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 995242]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[10344968, 948942, 15831422, 879504, 1332777]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1440974, 1014288, 1295300, 831863, 1008602]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[984724, 1067835, 913089, 15716429, 15737492]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 1098066]"
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8...","[9526903, 9245327, 2011357, 41019, 9676747]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 995242, 1098066]"
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884...","[1120837, 469192, 1148749, 2162110, 10461135]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 1127831]"
7,14,"[840601, 867293, 933067, 951590, 952408, 96569...","[1818418, 12187399, 862735, 10356468, 13213079]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 995242]"
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13...","[1068830, 1826563, 1070989, 34873, 1100601]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]"
9,16,"[1062973, 1082185, 13007710]","[16770135, 1170488, 29252, 13038888, 910378]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 995242, 1029743]"


### 4.2 Косинусное сходство и CosineRecommender

![cosine_similarity.png](attachment:cosine_similarity.png)

In [32]:
%%time

model = CosineRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.81 s, sys: 11.2 ms, total: 1.82 s
Wall time: 1.27 s


In [33]:
[id_to_itemid[rec[0]] for rec in recs]

[1082185, 1029743, 1081177, 904360, 6034857]

In [34]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 50.4 ms, sys: 0 ns, total: 50.4 ms
Wall time: 54.4 ms


In [35]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13910406, 13192522, 62804, 1131236, 1042390]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 1029743, 1081177, 904360, 6034857]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[9803602, 959210, 929171, 13987234, 12525209]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 951590, 1044078, 9526410, 904360]"


### 4.3 TF-IDF взвешивание и TFIDFRecommender

![tf_idf.png](attachment:tf_idf.png)

Если 2 юзера оба купили очень популярный товар, то это еще не значит,что они похожи   
Если 2 юзера оба купили редкий товар, то они похожи

Занижаем вес популярных товаров при расчете расстояний между пользователями

In [36]:
%%time

model = TFIDFRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.84 s, sys: 17.2 ms, total: 1.85 s
Wall time: 1.31 s


In [37]:
[id_to_itemid[rec[0]] for rec in recs]

[1082185, 981760, 1127831, 6666, 1098066]

In [38]:
%%time

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=False, 
                                    recalculate_user=False)])

CPU times: user 71.3 ms, sys: 0 ns, total: 71.3 ms
Wall time: 71.7 ms


In [39]:
result.head(5)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13910406, 13192522, 62804, 1131236, 1042390]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 1029743, 1081177, 904360, 6034857]","[1082185, 981760, 1127831, 6666, 1098066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[9803602, 959210, 929171, 13987234, 12525209]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 951590, 1044078, 9526410, 904360]","[1082185, 981760, 1098066, 826249, 6666]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[10344968, 948942, 15831422, 879504, 1332777]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 951590, 994928, 1029743, 1044078]","[1082185, 981760, 1127831, 6666, 878996]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1440974, 1014288, 1295300, 831863, 1008602]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 994928, 1044078, 1029743, 6034857]","[1082185, 981760, 1127831, 6666, 961554]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[984724, 1067835, 913089, 15716429, 15737492]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 1098066]","[1082185, 854852, 951590, 1029743, 1044078]","[1082185, 981760, 6666, 1098066, 826249]"


### 4.4 Трюк

In [40]:
%%time

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей


model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.68 s, sys: 12.4 ms, total: 1.69 s
Wall time: 1.14 s


In [41]:
%%time

result['own_purchases'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[6666]], 
                                    recalculate_user=True)])

CPU times: user 49.8 ms, sys: 1.96 ms, total: 51.7 ms
Wall time: 53.6 ms


### 4.5 Измерим качество по precision@5

In [42]:
result.head(5)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13910406, 13192522, 62804, 1131236, 1042390]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 1029743, 1081177, 904360, 6034857]","[1082185, 981760, 1127831, 6666, 1098066]","[1082185, 995242, 1029743, 840361, 904360]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[9803602, 959210, 929171, 13987234, 12525209]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 951590, 1044078, 9526410, 904360]","[1082185, 981760, 1098066, 826249, 6666]","[1082185, 1098066, 6534178, 826249, 1127831]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[10344968, 948942, 15831422, 879504, 1332777]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 951590, 994928, 1029743, 1044078]","[1082185, 981760, 1127831, 6666, 878996]","[1082185, 981760, 995242, 1029743, 840361]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1440974, 1014288, 1295300, 831863, 1008602]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 994928, 1044078, 1029743, 6034857]","[1082185, 981760, 1127831, 6666, 961554]","[1082185, 995242, 1029743, 826249, 1127831]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[984724, 1067835, 913089, 15716429, 15737492]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 1098066]","[1082185, 854852, 951590, 1029743, 1044078]","[1082185, 981760, 6666, 1098066, 826249]","[1082185, 981760, 995242, 1029743, 840361]"


In [46]:

import os, sys
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [48]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [49]:
def average_precision_at_k(recommended_matrix, bought_matrix, k=5):
    
    rows_count = bought_matrix.shape[0]
    precisions_by_row = [precision_at_k(recommended_matrix[i], bought_matrix[i], k) for i in range(rows_count)]
    precisions_mean = np.mean(precisions_by_row)

    return precisions_mean

columns = result.columns.drop(['user_id', 'actual'])

for column in columns:
    precisions_mean = average_precision_at_k(result[column], result.actual, k=5)
    print('{:35} {:.4f} %'.format(column, precisions_mean * 100))

random_recommendation               0.0392 %
popular_recommendation              15.5240 %
itemitem                            13.6827 %
cosine                              17.2764 %
tfidf                               13.8981 %
own_purchases                       nan %


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in long_scalars
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
# your_code

Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей  K .

In [50]:
result_user_item = []
k_list = np.arange(1, 11)

for k in k_list:
    model = ItemItemRecommender(K=k, num_threads=4)
    model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)
    recomended_matrix = []
    for userid in result.user_id:
        recomends = model.recommend(userid=userid_to_id[userid], 
                                    user_items=sparse_user_item,
                                    N=5, 
                                    filter_already_liked_items=True, 
                                    filter_items=None, 
                                    recalculate_user=True)
        recomended_matrix.append([id_to_itemid[rec[0]] for rec in recomends])

    precisions_mean = average_precision_at_k(recomended_matrix, result.actual, k=5)
    result_user_item.append(precisions_mean)

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

Вывод: При K=5 видим результат как и раньше около 3,5%. Но лучший результат достигается при уменьшении числа ближайших соседей до 2.

Мне кажется что предпочтения в покупках не должны зависеть от разбиения датафрейма. Скажите, пожалуйста, это правильный подход?